In [2]:
import anndata as ad
import pandas as pd
import scanpy as sc
import numpy as np
import scipy as scp
import os
import re
import collections as cl

In [3]:
#Read the downloaded output file names and combine file with same S
adataNames = os.listdir('./adataObjs/')
adataList = []
adataNames.sort()
#adataNames

FileNotFoundError: [Errno 2] No such file or directory: './adataObjs/'

In [4]:
phenoDat = pd.read_csv('phenoSCCall.csv')
phenoDat.drop('Unnamed: 0',axis=1, inplace=True)
print(phenoDat.shape)
phenoDat.head()

(23914, 4)


,cell,treatment,replicate,barcodes
0,SCC1,CTX,R1,AAACCTGAGTTGTCGT
1,SCC1,CTX,R1,AAACCTGCACAGCGTC
2,SCC1,CTX,R1,AAACCTGGTACTTCTT
3,SCC1,CTX,R1,AAAGATGGTGTGAATA
4,SCC1,CTX,R1,AAAGCAAGTAGCTGCC


In [11]:
dup = phenoDat.duplicated('barcodes',keep=False)
dupPhenoDat = phenoDat.loc[dup,:]
dupPhenoDat.index

Int64Index([    4,    42,    61,   110,   117,   145,   173,   199,   245,
              250,
            ...
            23544, 23569, 23622, 23740, 23754, 23765, 23846, 23855, 23884,
            23890],
           dtype='int64', length=657)

In [5]:
dupPhenoDat.sort_values('barcodes').head()

,cell,treatment,replicate,barcodes
11621,SCC25,PBS,R2,AAACGGGAGAAGATTC
7273,SCC25,CTX,R2,AAACGGGAGAAGATTC
4,SCC1,CTX,R1,AAAGCAAGTAGCTGCC
965,SCC1,CTX,R2,AAAGCAAGTAGCTGCC
3616,SCC1,PBS,R2,AACACGTCATCAGTCA


In [9]:
dup2 = dupPhenoDat.duplicated(['barcodes','treatment'],keep=False)
dupKeep = dupPhenoDat.loc[dup2,:].sort_values('barcodes')
dupKeep.index

Int64Index([    4,   965,  9344, 11662, 14257,  5322, 21512, 11687, 18811,
             9373,
            ...
             3590, 14137, 18670,   937,  2317,   939, 14155, 21399, 21406,
            23890],
           dtype='int64', length=316)

In [14]:
remove = [x for x in list(dupPhenoDat.index) if x not in list(dupKeep.index)]

In [18]:
pD = phenoDat.drop(remove,60)
print(pD.shape)
len(set(pD['barcodes']))

(23573, 4)


23415

In [ ]:
keep = list(set(phenoDat['barcodes']))

In [ ]:
%%time
for filename in adataNames[1:]:
    print('started working on ' + filename)
    an = ad.read_h5ad('./adataObjs/'+filename)
    cX = an.X.toarray()
    cS = an.layers['spliced'].toarray()
    cU = an.layers['unspliced'].toarray()
    var_names = an.var.index.values
    delete = []
    obsNames = []
    print('checking indices')
    for i in range(an.shape[0]):
        if(i%25000 == 0):
            print('index reached' + str(i))
        if an.obs.index[i] not in keep:
            delete.append(i)
        else:
            obsNames.append(an.obs.index[i])
    print('deleting')
    cX = np.delete(cX,delete,0)
    cS = np.delete(cS,delete,0)
    cU = np.delete(cU,delete,0)
    print(obsNames[:10])
    cX = scp.sparse.csr_matrix(cX)
    cS = scp.sparse.csr_matrix(cS)
    cU = scp.sparse.csr_matrix(cU)
    updatedAdata = ad.AnnData(cX,
                      {'obs_names': obsNames},
                      {'var_names': var_names},
                       layers={'spliced':cS,
                              'unspliced':cU})
    fname = './adataObjsKeep/' + filename[7:-5] + '_keep'+ '.h5ad'
    print('updated adata is:')
    print(updatedAdata)
    print('writing file '+fname)
    updatedAdata.write(fname)